In [3]:
from google.colab import drive
drive.mount('/content/drive')
extract_path = '/content/food_dataset/'

TRAIN_CNN2 = os.path.join(extract_path, 'training')
VALID_CNN2 = os.path.join(extract_path, 'validation')
TEST_CNN2 = os.path.join(extract_path, 'testing')
MODELS_DIR = '/content' 

Mounted at /content/drive


NameError: name 'os' is not defined

In [4]:
!unzip -q "/content/drive/MyDrive/dataset1.zip" -d "/content/dataset1/"

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from copy import deepcopy

NUM_CLASSES = 82
NUM_EPOCHS = 80

import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
extract_path = '/content/dataset1/split_dataset'


# Postavi dataset putanje
TRAIN_CNN2 = os.path.join(extract_path, 'training')
VALID_CNN2 = os.path.join(extract_path, 'validation')
TEST_CNN2 = os.path.join(extract_path, 'testing')
MODELS_DIR = '/content'

# ----- Data transforms -----
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.ImageFolder(TRAIN_CNN2, transform=train_transform)
valid_dataset = datasets.ImageFolder(VALID_CNN2, transform=valid_transform)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ----- Model -----
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ----- Training loop -----
best_acc = 0.0
best_model_wts = None

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {running_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Save best
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = deepcopy(model.state_dict())
        os.makedirs(MODELS_DIR, exist_ok=True)
        torch.save(best_model_wts, os.path.join(MODELS_DIR, "best_model_cnn2.pth"))
        print(f"✅ Best model updated! Acc={best_acc:.2f}%")

Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 210MB/s]


Epoch 1/80, Loss: 3.1976
Validation Accuracy: 44.05%
✅ Best model updated! Acc=44.05%
Epoch 2/80, Loss: 2.2384
Validation Accuracy: 51.60%
✅ Best model updated! Acc=51.60%
Epoch 3/80, Loss: 1.9258
Validation Accuracy: 56.85%
✅ Best model updated! Acc=56.85%
Epoch 4/80, Loss: 1.7014
Validation Accuracy: 57.75%
✅ Best model updated! Acc=57.75%
Epoch 5/80, Loss: 1.5416
Validation Accuracy: 60.44%
✅ Best model updated! Acc=60.44%
Epoch 6/80, Loss: 1.4362
Validation Accuracy: 60.76%
✅ Best model updated! Acc=60.76%
Epoch 7/80, Loss: 1.3448
Validation Accuracy: 59.35%
Epoch 8/80, Loss: 1.2755
Validation Accuracy: 59.22%
Epoch 9/80, Loss: 1.2075
Validation Accuracy: 61.91%
✅ Best model updated! Acc=61.91%
Epoch 10/80, Loss: 1.1306
Validation Accuracy: 62.04%
✅ Best model updated! Acc=62.04%
Epoch 11/80, Loss: 1.0811
Validation Accuracy: 60.24%
Epoch 12/80, Loss: 1.0065
Validation Accuracy: 60.82%
Epoch 13/80, Loss: 0.9627
Validation Accuracy: 61.97%
Epoch 14/80, Loss: 0.8960
Validation Accura